# Groupassignment PCA (1)

J.P.A. (Juul) van Boxtel - 1019575  
B.W.M. (Bram) Geven - 1327909  
M.A. (Mirthe) Kamphuis - 1259725  
K.J.M. (Kirsten) Lukassen - 1228126  

This is the first Jupyter Notebook for the groupassignment of the course 8CC00 made by group 4. This notebook guides the reader through all different methods that were created to perform a principal component analysis of the RMA expression in cancer cells. The documentation is mainly in the code and will therefore not be repeated in the text of the notebook, only additional information and a complexity analysis is given. The code for the results and a brief discussion of them can be seen in a [second notebook](./Groupassignment_group_4_results.ipynb).

To provide a shorter and clearer way to the results, a [seperate report](../8CC00-GroupassignmentPCA-Group04.pdf) has been written. Next to the results and a analysis of the results, the rebuttals to the evaluations of the other groupmembers are provided in this separate report.

The full code, including all modules and methods, can be found in the zip file. 

## Overview

The following class is present in this notebook:

- [CellLineRMAExpression](#CellLineRMAExpression)

The following methods are present in this notebook:

- [initclassvars](#initclassvars)
- [load_RMAExp_to_CellLines](#load_RMAExp_to_CellLines)
- [load_RMAExp_to_matrix](#load_RMAExp_to_matrix)
- [normalize_list](#normalize_list)
- [normalize_matrix](#normalize_matrix)
- [covariance_of_two_lists](#covariance_of_two_lists)
- [covariance_matrix](#covariance_matrix)
- [calcMaxIdx](#calcMaxIdx)
- [getMaxIdxs](#getMaxIdxs)

Of all of these, a brief explanation will be given together with a complexity analysis. The class and methods will be runned in the [notebook with all results](./Groupassignment_group_4_results.ipynb). 


But, first, the math and numpy libraries need te be imported.

In [2]:
import math
import numpy as np

### CellLineRMAExpression

First, a class named `CellLineRMAExpression` was created. This class is used to store the RMA expression information of cell lines. An instance of this class represents a specific cell line, of which various information is known: the name of the cell line, cosmic id and the cancer type. So at initialisation, these variables are assigned to the instance, which happens at the initialisation method of the class. If no values are given, default values for *CellLineName*, *CosmicID* and *CancerType* are assigned. 

In the class, the method `load_RMAExpression` is added. This method adds a dictionary 'RMAExp_dict' to the instance of the class. It then iterates over the gene labels of 'allparskeys' and assigns the corresponding given value. At default, the method fills the dictionary with zeros. 

In [3]:
class CellLineRMAExpression:
    """
    Class used to store the RMA expression values of a cell line.
    """
    allparskeys = []
    
    def __init__(self, CellLineName = 'AU565', CosmicID = '910704', CancerType='BRCA'):
        """
        Initiate the values of an instance of class CellLineRMAExpression
        
        Parameters:
            CellLineName, string containing the name of this cellline 
            CosmicID, string containing the Cosmic ID of this cellline
            CancerType, string containing the Cancer type of this cellline
        """
        self.CellLineName = CellLineName #assign given name to instance
        self.CosmicID = CosmicID #assign given cosmic ID to instance
        self.CancerType = CancerType #assign given cancer type to instance 
        
    def load_RMAExpression(self, values = [0]*len(allparskeys)):
        """
        Given a list of numbers with the same length as allparskeys, values,
        assign the given values to the corresponding genes in this instance, 
        and store it in a dictionary with the gene names as keys. 
        
        Parameters:
            values, list of 244 numbers representing the RMAExpression of the genes defined in allparskeys of this instance
        """
        self.RMAExp_dict={} #create empty dictionary to store RMAExpression values in for this instance
        for i in range(len(CellLineRMAExpression.allparskeys)): #loop over all genes specified in allparskeys
            self.RMAExp_dict[CellLineRMAExpression.allparskeys[i]]=values[i] #assign corresponding values to the genes

The iterating process is the most time consuming. By growing the dictionary (adding elements at each iteration) there is a risk that the memory where the dictionary is initially assigned to does not suffice, and that it has to be reallocated or divided. When always working with the same number of genes, this dictionary could be predefined, which would prevent this from happening.

### initclassvars 

The `initclassvars` method assigns the elements of a list to a new list. 

In [4]:
def initclassvars(l):
    """
    Set the parameters names in class CellLineRMAExpression to the elements of l
    
    Parameter: l non-empty list
    """
    CellLineRMAExpression.allparskeys=l

The complexity is linear to the number of keys which to be initialized. 

### load_RMAExp_to_CellLines

The method `load_RMAExp_to_CellLines` requires the metadata of the cell lines and the corresponding rma expression as input, in the format of a pandas dataframe. It is optional to specify a list of indexes, which selects the cell lines to use from metadata. If no list is given, all cell lines are used. 

First, the names of the genes are assigned to the *allparskeys* list of class `CellLineRMAExpression` by method `initclassvars`. For each index in *ListOfCellLineNumbers*, the cell line information is extracted from metadata (e.g. name, cosmic id, TCGA label). The extracted name is then used to find the RMAExpression of this cell line. An instance is then created, and the metadata is assigned to this instance at initialisation. The RMAExpression values are then assigned using the
`load RMAExpression` method. The instance is then stored in a list. This is repeated for each index, after which the list with all instances is returned.

In [5]:
def load_RMAExp_to_CellLines(metadata, rma_expr, ListOfCellLineNumbers = None, metadata_labels = ["name", "COSMIC_ID", "TCGA_label"], lookup_variable = "name"):
    """
    Create a list of instances of class 'CellLineRMAExpression' per cell line given in ListOfCellLineNumbers, 
    and fill each instance with the correct name, cosmic_id, TCGA_label from metadata and the corresponding 
    RMAExpression values from rma_expr based on the lookup_variable.  
        
    Parameters: 
        metadata, non-empty pandas dataframe containing the name, cosmic ID and TCGA label (columns) of the cell lines (rows)
        rma_expr, non-empty pandas dataframe containing the RMAExpression per gene (columns) of each cell line (rows), 
                  indexed with the lookup_variable of the cell line
        ListOfCellLineNumbers, non-empty list of indexes corresponding to dataframe metadata
        metadata_labels, list of three strings, representing the names of the columns in metadata with the name, 
                         cosmid_ID and TCGA_label information (in that order)
        lookup_variable, string containing one of: "name", "cosmic_id", "tcga_label", 
                         representing the variable to which to match metadata and rma_expr
        
    Returns: a list of instances of the CellLineRMAExpression class, filled with the corresponding information and RMAExpression
    """
    initclassvars(rma_expr.columns) #assign gene names to class variable 'allparskeys'

    if ListOfCellLineNumbers is None: ListOfCellLineNumbers = metadata.index #if no indexes are specified, use all indexes
    nr_instances = len(ListOfCellLineNumbers) #Get the number of instances to create
    List_of_cellline_classes = [None]*nr_instances #Initialize shape size to prevent reallocation of memory
    
    for i, idx in enumerate(ListOfCellLineNumbers):  #Iterate over every cell line index, with idx the correct index and i the iteration
        name, cosmic_id, tcga_label = metadata.loc[idx, metadata_labels] #Get the name, cosmic_id and tcga_label of the cell line with index idx
        values = rma_expr.loc[str(eval(lookup_variable))].values #Get the RMA Expression values of the cell line with an index corresponding to 'name', the name of cell line i
    
        instance = CellLineRMAExpression(CellLineName=name, CosmicID=cosmic_id, CancerType=tcga_label) #load the cell line information into the instance
        instance.load_RMAExpression(values) #load the RMAExpression values into the instance
        List_of_cellline_classes[i] = instance #save the instance to the list which will be used as output 

    return List_of_cellline_classes

The loading and assigning of all data is the most time consuming. The merging of the metadata with the RMAExpression data is time consuming, since for every instance in metadata the corresponding name in RMAExpression needs to be found. For M cell lines and N RMAExpression data lines for P genes, the number of steps could approach MxN, which can grow rapidly by increasing the sizes of the datasets. Then for M cell lines, also P values are to be assigned to the instance. There a linear growth is seen when the number of genes increases.

### load_RMAExp_to_matrix

The method `load RMAExp to matrix` takes the list of instances as input. A MxN matrix is constructed, with M the number of instances and N the number of genes per instance. Then for each Mth instance, the N values of the RMAExp dict are extracted, and placed into row M of the matrix. The MxN numpy matrix is then returned.

In [7]:
def load_RMAExp_to_matrix(data):
    """
    Given a list of CellLineRMAExpression class instances,
    Create an MxN matrix containing the RMA Expression values, with M the number of instances and N the number of genes
        
    Parameters: 
        data, non-empty list of CellLineRMAExpression class instances filled with the RMA Expression values 
              stored in the RMAExp_dict attribute of the instance
        
    Returns: a MxN numpy array containing the RMA Expression values, with M the number of instances and N the number of genes
    """
    nr_instances = len(data) #get the number of instances
    nr_variables = len(data[0].RMAExp_dict) #get the number of genes 
    rma_exp_matrix = np.empty((nr_instances, nr_variables)) #Initialize empty numpy array to prevent reallocation of memory
    
    for i, instance in enumerate(data): #iterate over every instance, with i the iteration
        rma_exp_matrix[i] = list(instance.RMAExp_dict.values()) #extract the values of the RMAExp_dict of the instance, containing the RMAExpression values, and store them in the matrix
    
    return rma_exp_matrix

In this method, the steps to be taken scale linearly with the number of cell lines and genes. Therefore, when more cell lines and/or genes are present in the data, it takes longer to load the data. 

### normalize_list

The method `normalize_list` iterates over all columns in the matrix, then calculates the mean and standard deviation of this column. Then for each element in the column, the mean is subtracted and is then divided by the standard deviation.

In [8]:
def normalize_list(variable_list):
    """
    Given a list of numbers,
    create a list with its normalized values
        
    Parameters: 
        variable_list, a non-empty list of numbers 
        
    Returns: a list containing the z-score normalized values of data
    """
    mean = sum(variable_list) / len(variable_list) #get the mean of the variable
    variance = sum(pow(x-mean,2) for x in variable_list) / len(variable_list) #get the variance of the variable
    std = math.sqrt(variance) #get the standard deviation of the variable
    
    normalized_list = [(x-mean)/std for x in variable_list] #subtract the mean and divide by the standard deviation for every element of the list
   
    return normalized_list

It is a quite costly method, as for N columns of length M, M+1 steps need to be taken for the mean. Then for the standard deviation for each element the mean needs to be subtracted, the result should be squared, and all results should be summed. Then the square root should be taken. This results in 3*M + 2 steps, and that N times.

### normalize_matrix

The method `normalize_matrix` creates a MxN numpy array with normalized values of the data per column N. In this method, another method `normalize_list` is used. At the end, a MxN numpy array containing the z-score normalized values of the data is returned. 

In [9]:
def normalize_matrix(data_matrix):
    """
    Given a MxN numpy array,
    create a MxN numpy array containing the z-score normalized values of data per column N
        
    Parameters: 
        data, a non-empty MxN numpy array of numbers
        
    Returns: a MxN numpy array containing the z-score normalized values of data
    """
    data_matrix_norm = data_matrix.copy() #copy the matrix, so the original is not overwritten
    nr_variables = data_matrix_norm.shape[1] #get the number of variables: N
    
    for i in range(nr_variables): #iterate over every variable
        data_matrix_norm[:,i] = normalize_list(data_matrix_norm[:,i]) 
   
    return data_matrix_norm

The method `normalize_matrix` scales linearly with the number of variables. Since the method `normalize_list` is used in this method, the number of steps also depend on this method. The complexity of `normalize_list` is explained in the previous section.

### covariance_of_two_lists

The method `covariance_of_two_lists` takes two lists of numbers of the same length and calculates their covariance. This is done by calculating the mean of each liss, which was executed by taking the sum and dividing this by the number of elements. Then an element wise multiplication for both lists is made, while corrected for their means. The covariance is then calculated by taking the mean of this result. The equation can be seen below. 

$$Covariance =\frac{{\sum}(x[i]-\overline{x})(y[i]-\overline{y})}{N-1}$$
Source: [Investopedia](https://www.investopedia.com/terms/c/covariance.asp) 

In [10]:
def covariance_of_two_lists(x, y):
    """
    Given two lists of numbers of equal length,
    calculate their covariance
        
    Parameters: 
        x, a non-empty list of normalized numbers
        y, a non-empty list of normalized numbers of equal length as x
        
    Returns: the covariance of x and y
    """
    
    cov_list = [a * b for (a,b) in zip(x,y)] #make an element wise multiplication
    covariance = sum(cov_list) / (len(cov_list)-1) #calculate the covariance
    
    return covariance

The number of steps that need to be taken is twice the number of elements in the lists, once when performing an element wise multiplication and again when computing the sum. With computing the length of the *cov_list*, another step is added.

### covariance_matrix

After normalizing the data, the covariance matrix can be computed. Therefore the method `covariance matrix` is created, which takes the normalized matrix as input. First an empty matrix of NxN is constructed. Then the method iterates over every element of the matrix and calculates the covariance. Because of the symmetry of the covariance matrix, the known covariance of y,x is also assigned to the x,y position in the covariance matrix. Then, the covariance matrix is returned. 

In [11]:
def covariance_matrix(data):
    """
    Given a MxN numpy array of numbers, with M the number of instances and N the number of variables
    Calculate the NxN covariance matrix 
        
    Parameters: 
        data, a non-empty MxN numpy array of normalized numbers, with M observations and N featueres
        
    Returns: a NxN numpy array containing the covariance matrix of the data
    """
    nr_variables = data.shape[1] #get the number of variables N
    covMatrix = np.zeros((nr_variables, nr_variables)) #Initialize empty numpy array to prevent reallocation of memory

    for x in range(nr_variables): #iterate over the number of variables 
        for y in range(x, nr_variables): #iterate over the number of variables 
            covMatrix[x, y] = covariance_of_two_lists(data[:,x], data[:,y]) #Calculate the covariance of variables X and Y
            covMatrix[y, x] = covMatrix[x, y] #Assign the known covariance of Y, X to X, Y position in the covariance matrix
    
    return covMatrix

Since the covariance matrix is an NxN matrix, the steps to be taken scale exponentially with a growing number of variables. Because the matrix is symmetrical, less values need te be computed. Therefore the steps to be taken actually scale exponentially wiht a growing number of variables times 0.5. However, since all values need to be allocated to the memory, it still takes some extra time. 

When comparing the results of the covariance matrix constructed by this method to the numpy method, no differences are seen.

### calcMaxIdx 

The method `calcMaxIdx` returns the index of the maximum values of a list.

In [12]:
def calcMaxIdx(l=[1]):
    """
    Given a list
    find the index of the maximum value
    
    Parameters:
        l non-empty list of numbers or nan's, with at least one number
    
    Returns: the index of the maximum value
    """
    nx = 0 #set the index of the maximal value to 0
    mx=l[nx] #set the maximal value to the value at index nx
    
    while(math.isnan(mx)): 
        #verify that the set maximal value is a number
        #if not, iterate till a number is found, and assign it as the maximal value
        nx += 1
        mx = l[nx]
        
    #mx is the current max
    n=0
    while (n<len(l)):
        # for all 0<=i<n: l[i]<=mx and
        # there exists an i, 0<=i<n such that l[i]=mx
        # check if the element is not a nan
        if not math.isnan(l[n]) and l[n]>mx: 
            mx=l[n]
            nx=n
        n=n+1
        # for all 0<=i<n: l[i]<=mx and
        # there exists an i, 0<=i<n such that l[i]=mx
        
    # n=len(l) and for all 0<=i<n: l[i]<=mx and
    # for all 0<=i<len(l): l[i]<=mx and
    # there exists an i, 0<=i<len(l) such that l[i]=mx
    # hence, this i = nx
    
    return nx

The complexity of `calcMaxIdx` is of the order of the length of the list. In the first while loop, the number of iterations depends on the number of NaN values before the first number. In the second while loop, the maximum number of steps taken is the length of the input list. 

### getMaxIdxs 

The method `getMaxIdxs` takes a list of values as input, and an integer N, and returns a list with the indexes of the N maximal
values of the list. The indexes are sorted in descending order of their corresponding max values, thus the index of the maximum value comes first, the next maximum second etc. A list containing the indexes of the maximum values of the input list in order of highest value will be returned. 

In [14]:
def getMaxIdxs(l=[1], number_idxs=1):
    """
    Given a list
    find the indexes of the number_idxs maximum values, in order from highest to lowest
    
    Parameters: 
        l, non-empty list of numbers or nan's, with at least number_idxs numbers
           number_idxs, integer representing the amount of indexes to find
    
    Returns: a list containing the indexes of the maximum values of list l, in order of highest value
    """
    max_idxs = [None]*number_idxs #Initialize empty list to prevent reallocation of memory when saving results
    control_list = np.asarray(l.copy()).real.astype(float) #Copy the original list to prevent overwriting, and make sure the numbers are all real numbers of float type
    
    for i in range(number_idxs): #find the max index 'number_idxs' times
        idx = calcMaxIdx(control_list) #find the max index
        max_idxs[i] = idx #save the max index to the result list
        control_list[idx] = np.nan #change the value at the max index to nan so it won't be found again, thus the next max index can be found
        
    return max_idxs

This method can become computationally more expensive then just sorting the list, when a high N is chosen. However, in PCA this is rarely the case since then the whole idea behind PCA goes lost (reducing the number of components).

### Results 

As stated before, the results can be viewed in the [report](../8CC00-GroupassignmentPCA-Group04.pdf) and in the [second notebook](./Groupassignment_group_4_results.ipynb). 